In [1]:
##transcirpt
import requests
import json
import openai
import os


def transcribe(audio):
    # audio_filename_with_extension = audio + '.wav'
    # os.rename(audio, audio_filename_with_extension)
    audio_file = open(audio, "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    return transcript["text"]


input_audio = "audio.mp3"
prompt = transcribe(input_audio)
output_text_file_path = "output.txt"
with open(output_text_file_path, "w") as text_file:
    text_file.write(prompt)


In [5]:
####transcript格式
import os
import re

def edit_txt_file(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    with open(output_file, 'w') as f:
        for line in lines:
            if line.strip():
                line.strip()
                line = re.sub(r'说话人(\d+)', lambda match: 'speaker' + match.group(1), line)
                line = re.sub(r'\(\d+:\d+:\d+\)', '', line).replace('speaker01', 'speaker1').replace('speaker02', 'speaker2')
                f.write(line)

meeting_id = 10
for file in os.listdir("transcript/group2"):
    if file.endswith(".txt"):
        output_file = f"transcript/group2new/{meeting_id}.txt"
        input_path = f"transcript/group2/{file}"
        edit_txt_file(input_path, output_file)
        meeting_id += 1

# file = "meeting_07.txt"
# output_file = f"transcript/{file}"
# input_path = f"transcript/raw/{file}"
# edit_txt_file(input_path, output_file)


In [9]:
## summarize
import openai
import os


text_type = "transcript"
meeting_number = "16"
output_answer_file_path = f'summary/group2/round0/round0_{meeting_number}.txt'
input_text_file_path = f'{text_type}/group2new/{meeting_number}.txt'
raw_transcript = open(input_text_file_path, "r").read()
instruction = f"""
为每个与会者定制一份中文摘要

每份摘要要求：
1.生成标题概括主题，标题前加上'# '号
2.概述会议的主要内容和待办事项
3.使用<br>作为换行符分割每份摘要内的要点
4.给某用户的摘要中，以第一人称代指该用户，所有人名前与后加上**符号

输出格式：
-speaker1
[摘要1]
-speaker2
[摘要2]

会议记录：
{raw_transcript}
"""

print(instruction)

completion = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "你是一名笔记助手，擅长总结会议记录。"},
        {"role": "user", "content": instruction}
    ],
    temperature=0
)
out =  completion.choices[0].message.content
with open(output_answer_file_path, "w") as text_file:
    text_file.write(out)


为每个与会者定制一份中文摘要

每份摘要要求：
1.生成标题概括主题，标题前加上'# '号
2.概述会议的主要内容和待办事项
3.使用<br>作为换行符分割每份摘要内的要点
4.给某用户的摘要中，以第一人称代指该用户，所有人名前与后加上**符号

输出格式：
-speaker1
[摘要1]
-speaker2
[摘要2]

会议记录：
speaker1: 真实电影。我们是放一个电影，你在干啥？让他们讨论一下。
speaker2: 我觉得。看它主题是一个班级电影交流会。他是不是可以？再有一些同学们，比如说分享自己感受或者一些。就是讨论交流的环节。
speaker1: OK. 我们要放电影吗？还是不用。
speaker2: 我觉得我们可以就是不用放一部整的电影，可能是提前收集一些大家比如说感兴趣的，或者希望分享的这种片段，然后来在交流的过程中去播放这些片段。
speaker1: ok也就是说。我们提前先调查一下他们。是。喜欢什么电影？然后再看一下投票率高的电影给放放映一下。类的。
speaker2: 我觉得可以。
speaker1: 可以的。
speaker2: 就比如说我们可以先大概确定一下这次活动的范围和时长。
speaker1: 范围是指。
speaker2: 比如说这只是咱们班的同学。还是。
speaker1: 应该是咱们班的吧，因为是什么班级电影交流会。
speaker2: Ok那就是限定在班级的同学。那时间的话可以是。比如说。周末晚上的两个小时。
speaker1: 周末晚上。可以。周末晚上七到九点。
speaker2: 可以，周末的七到九点。
speaker1: 然后。我们应该要搞一个什么主题吧？
speaker2: 推荐一下活动主题。
speaker1: 主题的话。
speaker1: 或者说我做个问卷。问一下。他们对什么主题感兴趣，先让他们先投个票。
speaker2: 你的意思就是我们在前期调查的时候？目光就是问同学你喜欢哪个影片，你可以去问他一下，问大家喜欢什么样的。对，就是说更大一点的类型，然后我们可以聚焦一个类型来讨论。
speaker1: 对。然后。或者说。影片的话，我们如果想的不够全面的话。可以调查问卷，让同学们投个稿，然后比如说他从哪个电影到哪个片段。然后我们筛选一下。比如说就是。这个问题就可以做成您对什么主题感兴趣，让他选了一个之后，

In [10]:
##splite speaker 1&2
def split_text(input_file, output_folder, meeting_number):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        if line.startswith("-speaker2"):
            speaker2_lines = lines[i+1:]
            speaker1_lines = lines[1:i]
            break
    else:
        speaker2_lines = []
        speaker1_lines = lines

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(f"{output_folder}/{meeting_number}_speaker1.txt", 'w') as f:
        f.write(''.join(speaker1_lines))
    with open(f"{output_folder}/{meeting_number}_speaker2.txt", 'w') as f:
        f.write(''.join(speaker2_lines))

for file in os.listdir("summary/group2/round0"):
    if file.endswith(".txt"):
        filename = file.split('.')[0]
        input_file = f"summary/group2/round0/{file}"
        output_folder = f"summary/group2/round0/{filename}"
        split_text(input_file,output_folder, filename)

In [67]:
##txt convert to md
def txt_to_md(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    md_content = ''.join(lines)

    with open(output_file, 'w') as f:
        f.write(md_content)

for i in range(0,4):
    input_folder= f'summary/new_transcript/split/user{i}/'
    for file in os.listdir(input_folder):
        if file.endswith(".txt"):
            input_file = f"{input_folder}/{file}"
            output_file = f"{input_folder}/{file.replace('.txt', '.md')}"
            txt_to_md(input_file, output_file)


In [14]:
###fixed md->txt
import markdown

def md_to_txt(input_file, output_file):
    with open(input_file, 'r') as f:
        md_content = f.read()

    # 使用markdown库将Markdown内容转换为纯文本
    txt_content = markdown.markdown(md_content)

    with open(output_file, 'w') as f:
        f.write(txt_content)

file_id = 'round1_12_speaker2'
meeting_id = 'round1_12'
round = 'round1'
input_file = f"summary/group2/{round}/{meeting_id}/{file_id}_fixed.md" 
output_file = f"summary/group2/{round}/{meeting_id}/{file_id}_fixed.txt"  

md_to_txt(input_file, output_file)
